In [71]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [72]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [73]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  100 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=91', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=현대로템&start=101', 'https://search.naver.com/search.n

100%|██████████| 1300/1300 [00:00<00:00, 1303824.77it/s]


In [74]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

100%|██████████| 300/300 [01:05<00:00,  4.55it/s]

검색된 기사 갯수: 총  1000 개

[뉴스 제목]
['현대로템, 폴란드 방산 분야 인재 양성 지원', '현대로템, 호주서 1.2조원 ‘잭팟’', '현대로템, 잇단 철도 수주로 탄력… 수소차량 개발도 박차', '수소로 움직이며 공기 정화도?...현대로템은 왜 트램에 공 들이나', '현대로템, 호주에 전동차 제작·공급한다…총 사업 규모 1.2조', "[다가오는 수소경제]'철도회사' 현대로템이 만드는 미래 모빌리티", '현대로템, 호주 퀸즐랜드 전동차 공급 사업 수주', '현대로템, 방사청과 2154억 규모 K-1 중구난차량 등 정비계약', 'KAI·한화에어로·현대로템, 韓·폴란드 방산 인재 키운다', '현대로템, 방위사업청과 2154원 규모 계약 체결', '현대로템, 호국보훈의 달 맞아 야전부대 전투장비 특별점검', '현대로템, 야전부대 전투장비 특별점검 추진', '현대로템, K2전차 등 야전부대 전투장비 600대 특별점검', '현대로템, 야전부대 전투장비 특별 점검', '현대로템, 야전부대 전투장비 특별점검 나선다', '“완성차처럼 방산 AS 강화”… 현대로템, 야전부대 전투장비 특별점검 추진', '현대로템, 야전 전력화 장비 특별 점검 캠페인 진행', '"현대로템, 18조 규모 신규 수주…중장기 성장세 탄탄"-키움', '키움證 "현대로템, K2 수출 가파른 성장 기대…목표가↑"', '부산 간 현대로템, 수소전기트램 첫 공개', "현대로템, 軍 '지휘소용 차량'…방사청과 7000억 공급 계약", 'GTX-A, 부산에 뜬다…현대로템, 국제철도기술산업전 참가', "현대로템 '수소전기트램' 실물 첫 공개", '현대로템, 미래 철도 패러다임 ‘H-Flow’ 첫선', '현대로템, 미래 철도 패러다임 ‘H-플로우’ 첫 선', '현대로템, 철도 전시회서 ‘수소전기트램’ 실물 최초 공개', '[클릭! 이 종목] 현대로템, 기관 매수세에 올해 41% `쑥`', '현대로템이 그린 미래 철도… 수소전기트램 실물 최초 공개', "동력 분산식 고속차량 '현대로템 EMU-320'",

In [75]:
###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\한화에어로스페이스\\{}_{}_page{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H%M%S'),page,page2),encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  174
